In [1]:
import re, time, json, pickle

import urllib
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

/home/tsuchiya/.local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
base_url = 'https://race.netkeiba.com'

# 開催日URL

In [3]:
# date_list = []

# for y in tqdm(range(2000, 2020+1)):
#     for m in range(1, 12+1):
#         url = f'{base_url}/top/calendar.html?year={y}&month={m}'
        
#         html = requests.get(url)
#         html.encoding = html.apparent_encoding 
#         soup = BeautifulSoup(html.text, "html.parser")
        
#         date_list += list(map(lambda x: x.get('href'), soup.find(class_='Calendar_Table').find_all('a')))
        
#         time.sleep(2 + np.random.rand())
        
# date_list = list(map(lambda x: x.replace('..', base_url), date_list))
# f = open('./data/date_list.pkl', 'wb')
# pickle.dump(date_list, f)

  0%|          | 0/21 [00:00<?, ?it/s]

In [4]:
f = open('./data/date_list.pkl','rb')
date_list = pickle.load(f)

# レースID

In [18]:
set(list(map(lambda x: ''.join(x.split('/')[:-2]), date_list)))

{'https:db.netkeiba.comracesum', 'https:race.netkeiba.com'}

In [13]:
date_list[:2]

['https://db.netkeiba.com/race/sum/06/20000105',
 'https://db.netkeiba.com/race/sum/08/20000105']

In [4]:
options = Options()
options.add_argument("--headless")

driver = webdriver.Firefox(options=options)

race_list = []

for url in tqdm(date_list):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'RaceTopRace')))
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    table = soup.find_all(class_='RaceList_Data')
    
    for t in table:
        race_list += list(map(lambda x: x.get('href'), t.find_all('a')))
    
    time.sleep(2 + np.random.rand())
    
race_list = list(map(lambda x: x.replace('..', base_url), race_list))
race_list = [x for x in race_list if 'movie' not in x]
f = open('./data/race_list.pkl', 'wb')
pickle.dump(race_list, f)

  0%|          | 0/3718 [00:00<?, ?it/s]

TimeoutException: Message: 
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:181:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:393:5
element.find/</<@chrome://remote/content/marionette/element.js:299:16


In [6]:
f = open('./data/race_list.pkl','rb')
race_list = pickle.load(f)

# レース詳細

In [7]:
# race_info = []
# horse_url = []

# for url in tqdm(race_list):
#     html = requests.get(url)
#     html.encoding = html.apparent_encoding 
#     soup = BeautifulSoup(html.text, "html.parser")
    
#     race_id = url.split('race_id=')[-1].split('&rf')[0]
#     year = race_id[:4]
    
#     # レース情報
#     race_dict = {}
#     race_dict['race_id'] = race_id
    
#     try: race_dict['place'] = soup.find(class_='RaceKaisaiWrap').find(class_='Active').text
#     except: race_dict['place'] = np.nan
        
#     race_dict['race_name'] = soup.find(class_='RaceName').text.strip()
#     race_dict['date'] = year + '年' + soup.find(id='RaceList_DateList').find(class_='Active').text
#     race_dict['race_num'] = soup.find(class_='RaceNumWrap').find(class_='Active').text.strip()
#     race_dict['start'] = list(map(lambda x: x.strip(), soup.find(class_='RaceData01').text.split('/')))[0]
#     race_dict['condition'] = list(map(lambda x: x.strip(), soup.find(class_='RaceData01').text.split('/')))[1]
#     race_dict['weather'] = list(map(lambda x: x.strip(), soup.find(class_='RaceData01').text.split('/')))[2]
#     race_dict['baba'] = list(map(lambda x: x.strip(), soup.find(class_='RaceData01').text.split('/')))[3]
    
#     table = soup.find_all(class_='Payout_Detail_Table')
#     for t in table:
#         for t2 in t.find_all('tr'):
#             label = t2.find('th').text
#             if label in ['複勝', 'ワイド']:
#                 value = t2.find_all('td')[1].text.split('円')[:-1]
#             else:
#                 value = t2.find_all('td')[1].text.strip()
#             race_dict[label] = value
    
#     race_info.append(race_dict)
    
#     # レース結果
#     try:
#         soup_table = soup.find(id='All_Result_Table').find('tbody').find_all('tr')
#     except:
#         time.sleep(2 + np.random.rand())
#         continue
        
#     r_list = []
#     for t1 in soup_table: 
#         t2 = t1.find_all('td')

#         r_dict = {}
#         for i, col in enumerate(['result', 'waku', 'umaban', 'horse_name', 'age', 'handicap', 'jockey', 'time',
#                                 'diff', 'popular', 'odds', '3F', 'corner', 'trainer', 'weight']):
#             r_dict[col] = t2[i].text.strip()
#             if col == 'horse_name': 
#                 hurl = t2[i].find('a').get('href')
#                 horse_url.append(hurl)
                
#         r_dict['horse_id'] = hurl.split('/')[-1]
#         r_list.append(r_dict)
        
#     pd.DataFrame(r_list).to_csv(f'./data/race/{race_id}.csv', index=None)
    
#     horse_url = list(set(horse_url))
#     time.sleep(2 + np.random.rand())
    
# pd.DataFrame(race_info).to_csv('./data/race_info.csv', index=None)

# f = open('./data/horse_url.pkl', 'wb')
# pickle.dump(horse_url, f)

In [8]:
f = open('./data/horse_url.pkl','rb')
race_list = pickle.load(f)

# ウマ情報

In [9]:
horse_info = {}

for url in tqdm(race_list):
    horse_id = url.split('/')[-1]
    horse_info['horse_id'] = horse_id
    
    html = requests.get(url)
    html.encoding = html.apparent_encoding 
    soup = BeautifulSoup(html.text, "html.parser")
    
    horse_info['name'] = soup.find(class_='horse_title').find('h1').text.strip()
    horse_info['detail'] = soup.find(class_='horse_title').find(class_='txt_01').text.strip().split('\u3000')
    horse_info['trainer'] = soup.find(summary='のプロフィール').find_all('td')[1].text
    horse_info['owner'] = soup.find(summary='のプロフィール').find_all('td')[2].text
    horse_info['sire'] = soup.find(class_='blood_table').find_all(class_='b_ml')[0].text.strip()
    horse_info['mare_sire'] = soup.find(class_='blood_table').find_all(class_='b_ml')[2].text.strip()
    
    race_result_list = []

    try: table = soup.find(class_='db_h_race_results').find('tbody').find_all('tr')
    except: pass

    for t in table:
        race_result = {}
        race_result['date'] = t.find_all('td')[0].text
        race_result['place'] = t.find_all('td')[1].text
        race_result['weather'] = t.find_all('td')[2].text
        race_result['name'] = t.find_all('td')[4].text
        race_result['waku'] = t.find_all('td')[7].text
        race_result['num'] = t.find_all('td')[8].text
        race_result['odds'] = t.find_all('td')[9].text
        race_result['pop'] = t.find_all('td')[10].text
        race_result['result'] = t.find_all('td')[11].text
        race_result['jockey'] = t.find_all('td')[12].text.strip()
        race_result['kinryou'] = t.find_all('td')[13].text
        race_result['dist'] = t.find_all('td')[14].text
        race_result['baba'] = t.find_all('td')[15].text
        race_result['time'] = t.find_all('td')[17].text
        race_result['sa'] = t.find_all('td')[18].text
        race_result['nobori'] = t.find_all('td')[22].text
        race_result['weight'] = t.find_all('td')[23].text
        race_result_list.append(race_result)

    pd.DataFrame(race_result_list).to_csv(f'./data/horse/{horse_id}.csv')
    
    time.sleep(2 + np.random.rand())
    
pd.DataFrame(horse_info).to_csv(f'./data/horse_info.csv')

  0%|          | 0/34995 [00:00<?, ?it/s]